# Anonimização Segmentada com face_anon_simple

Este notebook demonstra como usar o modelo `face_anon_simple` (diffusion) para anonimizar apenas regiões específicas do rosto, como olhos, boca, nariz, etc.

A anonimização é aplicada apenas nas partes segmentadas escolhidas, mantendo o resto da face inalterado.

## ⚠️ Importante para Google Colab

Se você estiver usando o Google Colab, certifique-se de que os diretórios `src/` e `utils/` estão disponíveis:

1. **Opção 1 - Upload manual**: Faça upload dos diretórios `src/` e `utils/` para o Colab
2. **Opção 2 - Git clone**: Se o projeto estiver no GitHub, use `!git clone <repo_url>`
3. **Opção 3 - Google Drive**: Monte o Google Drive e aponte para o diretório do projeto

O notebook tentará detectar automaticamente o caminho correto, mas você pode precisar ajustar manualmente se necessário.


## 🔧 Configuração do Ambiente (Opcional - apenas para Colab)

Se você estiver no Google Colab e os arquivos não estiverem disponíveis, use uma das opções abaixo:


In [ ]:
# ============================================
# OPÇÃO 1: Clonar do GitHub (se o projeto estiver no GitHub)
# ============================================
# Descomente e ajuste a URL do repositório:
# !git clone https://github.com/seu-usuario/face_anon_simple.git
# %cd face_anon_simple

# ============================================
# OPÇÃO 2: Montar Google Drive
# ============================================
# Se seus arquivos estão no Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/caminho/para/face_anon_simple

# ============================================
# OPÇÃO 3: Upload manual via interface do Colab
# ============================================
# Use o painel lateral do Colab para fazer upload dos diretórios src/ e utils/

print("⚠️ Esta célula é opcional. Execute apenas se precisar configurar o ambiente no Colab.")


In [2]:
!pip install face_alignment

In [10]:
import torch
from transformers import CLIPImageProcessor, CLIPVisionModel
from diffusers import AutoencoderKL, DDPMScheduler
from diffusers.utils import load_image, make_image_grid
import face_alignment
from PIL import Image
import numpy as np

from src.diffusers.models.referencenet.referencenet_unet_2d_condition import ReferenceNetModel
from src.diffusers.models.referencenet.unet_2d_condition import UNet2DConditionModel
from src.diffusers.pipelines.referencenet.pipeline_referencenet import StableDiffusionReferenceNetPipeline

from utils.segmented_anonymization import anonymize_faces_segmented


ModuleNotFoundError: No module named 'src'

## 1. Carregar Modelos


In [ ]:
face_model_id = "hkung/face-anon-simple"
clip_model_id = "openai/clip-vit-large-patch14"
sd_model_id = "stabilityai/stable-diffusion-2-1"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"Device: {device}, dtype: {dtype}")

print("Carregando UNet...")
unet = UNet2DConditionModel.from_pretrained(
    face_model_id, subfolder="unet", use_safetensors=True
)

print("Carregando ReferenceNet...")
referencenet = ReferenceNetModel.from_pretrained(
    face_model_id, subfolder="referencenet", use_safetensors=True
)

print("Carregando Conditioning ReferenceNet...")
conditioning_referencenet = ReferenceNetModel.from_pretrained(
    face_model_id, subfolder="conditioning_referencenet", use_safetensors=True
)

print("Carregando VAE...")
vae = AutoencoderKL.from_pretrained(
    sd_model_id, subfolder="vae", use_safetensors=True
)

print("Carregando Scheduler...")
scheduler = DDPMScheduler.from_pretrained(
    sd_model_id, subfolder="scheduler", use_safetensors=True
)

print("Carregando CLIP...")
feature_extractor = CLIPImageProcessor.from_pretrained(
    clip_model_id, use_safetensors=True
)
image_encoder = CLIPVisionModel.from_pretrained(
    clip_model_id, use_safetensors=True
)

print("Criando pipeline...")
pipe = StableDiffusionReferenceNetPipeline(
    unet=unet,
    referencenet=referencenet,
    conditioning_referencenet=conditioning_referencenet,
    vae=vae,
    feature_extractor=feature_extractor,
    image_encoder=image_encoder,
    scheduler=scheduler,
)
pipe = pipe.to(device, dtype=dtype)

print("✓ Pipeline carregado com sucesso!")


## 2. Inicializar Face Alignment


In [ ]:
fa = face_alignment.FaceAlignment(
    face_alignment.LandmarksType.TWO_D,
    face_detector="sfd",
    device=device
)
print("✓ Face Alignment inicializado")


## 3. Carregar Imagem de Teste


In [ ]:
image_path = "my_dataset/test/00482.png"
original_image = load_image(image_path)
print(f"✓ Imagem carregada: {original_image.size}")

# Visualizar imagem original
display(original_image)


## 4. Definir Casos de Teste

Definimos diferentes combinações de características faciais para testar a anonimização segmentada.


In [ ]:
test_cases = [
    {
        'features': ['eyes'],
        'name': 'Apenas Olhos',
        'num_steps': 30,  # Menos passos para teste mais rápido
    },
    {
        'features': ['mouth'],
        'name': 'Apenas Boca',
        'num_steps': 30,
    },
    {
        'features': ['eyes', 'mouth'],
        'name': 'Olhos + Boca',
        'num_steps': 30,
    },
    {
        'features': ['nose'],
        'name': 'Nariz',
        'num_steps': 30,
    },
    {
        'features': ['eyebrows', 'eyes'],
        'name': 'Sobrancelhas + Olhos',
        'num_steps': 30,
    },
    {
        'features': ['eyes', 'nose', 'mouth'],
        'name': 'Olhos + Nariz + Boca',
        'num_steps': 30,
    },
]

print(f"Total de casos de teste: {len(test_cases)}")
for i, case in enumerate(test_cases):
    print(f"  {i+1}. {case['name']}: {case['features']}")


## 5. Executar Anonimização Segmentada

Agora vamos aplicar a anonimização usando `face_anon_simple` apenas nas regiões segmentadas escolhidas.


In [ ]:
results = []
generator = torch.manual_seed(42)

for i, test_case in enumerate(test_cases):
    print(f"\n{'='*70}")
    print(f"Teste {i+1}/{len(test_cases)}: {test_case['name']}")
    print(f"Features: {test_case['features']}")
    print(f"Passos de inferência: {test_case['num_steps']}")
    print(f"{'='*70}")
    
    try:
        anon_image = anonymize_faces_segmented(
            image=original_image,
            face_alignment_model=fa,
            mask_features=test_case['features'],
            operator_type='diffusion',
            pipe=pipe,
            generator=generator,
            num_inference_steps=test_case['num_steps'],
            guidance_scale=4.0,
            anonymization_degree=1.25,
            dilate_radius=3,
            smooth_edges=True,
        )
        
        # Salvar resultado
        safe_name = test_case['name'].replace(' ', '_').replace('+', '_').lower()
        output_path = f"test_diffusion_segmented_{i+1:02d}_{safe_name}.png"
        anon_image.save(output_path)
        
        print(f"✓ Anonimização concluída")
        print(f"✓ Salvo em: {output_path}")
        
        # Visualizar resultado
        display(anon_image)
        
        results.append({
            'name': test_case['name'],
            'features': test_case['features'],
            'output': output_path,
            'image': anon_image,
            'status': 'success'
        })
        
    except Exception as e:
        print(f"✗ Erro: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            'name': test_case['name'],
            'status': 'error',
            'error': str(e)
        })


## 6. Resumo e Comparação

Criamos um grid com todos os resultados para comparação visual.


In [ ]:
# Resumo dos resultados
successful = [r for r in results if r.get('status') == 'success']
failed = [r for r in results if r.get('status') == 'error']

print("=" * 70)
print("RESUMO DOS TESTES")
print("=" * 70)
print(f"\n✓ Sucessos: {len(successful)}/{len(test_cases)}")
print(f"✗ Falhas: {len(failed)}/{len(test_cases)}")

if successful:
    print("\n✓ Testes concluídos com sucesso:")
    for r in successful:
        print(f"  - {r['name']}: {r['output']}")
        print(f"    Features: {r['features']}\n")

if failed:
    print("\n✗ Erros encontrados:")
    for r in failed:
        print(f"  - {r['name']}: {r.get('error', 'Erro desconhecido')}")


In [ ]:
# Criar grid de comparação
if len(successful) > 0:
    result_images = [r['image'] for r in successful]
    
    # Adicionar imagem original no início
    result_images.insert(0, original_image)
    
    grid = make_image_grid(result_images, rows=2, cols=4)
    grid.save("test_diffusion_segmented_comparison.png")
    
    print("✓ Grid de comparação criado!")
    display(grid)


## Nota Importante

A anonimização com `face_anon_simple` foi aplicada **apenas nas regiões segmentadas especificadas**. O resto da face permanece inalterado, demonstrando anonimização seletiva.

Isso permite:
- Anonimizar apenas características específicas (ex: apenas olhos)
- Manter outras partes do rosto visíveis
- Combinar diferentes características (ex: olhos + boca)
- Controlar precisamente quais partes são anonimizadas
